# Esercitazione Pratica: Rilevamento di Anomalie

## Corso di Machine Learning: Apprendimento Non Supervisionato

Questa esercitazione pratica ti guiderà attraverso l'implementazione e l'applicazione delle principali tecniche di rilevamento di anomalie discusse nelle lezioni teoriche. Esploreremo diversi dataset e vedremo come applicare e valutare vari metodi per identificare osservazioni anomale.

### Obiettivi dell'esercitazione:
- Implementare e applicare approcci statistici, metodi basati sulla densità, Isolation Forest, One-Class SVM e altri algoritmi di rilevamento anomalie
- Visualizzare e interpretare i risultati del rilevamento di anomalie
- Valutare le prestazioni dei diversi algoritmi
- Confrontare le diverse tecniche su vari tipi di dataset
- Applicare il rilevamento di anomalie a casi di studio reali

## Configurazione dell'ambiente

Iniziamo importando le librerie necessarie per questa esercitazione.

In [ ]:
# Importazione delle librerie necessarie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.model_selection import train_test_split
from scipy import stats
import time
import warnings
warnings.filterwarnings('ignore')

# Impostazioni di visualizzazione
plt.style.use('seaborn-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Parte 1: Generazione e Preparazione dei Dataset

Creiamo diversi dataset sintetici con anomalie per testare i vari algoritmi.

In [ ]:
# Funzione per visualizzare i dataset
def plot_dataset(X, y=None, title="Dataset", anomaly_label=1, alpha=0.7):
    plt.figure(figsize=(10, 8))
    
    if X.shape[1] > 2:
        # Se il dataset ha più di 2 dimensioni, utilizziamo PCA per visualizzarlo
        pca = PCA(n_components=2)
        X_2d = pca.fit_transform(X)
        plt.title(f"{title} (PCA 2D)\nVarianza spiegata: {pca.explained_variance_ratio_.sum():.2f}", fontsize=14)
    else:
        X_2d = X
        plt.title(title, fontsize=14)
    
    if y is not None:
        # Visualizzare punti normali e anomalie con colori diversi
        normal_mask = (y != anomaly_label)
        anomaly_mask = (y == anomaly_label)
        
        plt.scatter(X_2d[normal_mask, 0], X_2d[normal_mask, 1], 
                   c='blue', label='Normale', s=50, alpha=alpha)
        plt.scatter(X_2d[anomaly_mask, 0], X_2d[anomaly_mask, 1], 
                   c='red', label='Anomalia', s=80, alpha=alpha, marker='X')
        plt.legend()
    else:
        plt.scatter(X_2d[:, 0], X_2d[:, 1], s=50, alpha=alpha)
    
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# 1. Dataset con anomalie globali (outlier distanti dalla massa principale dei dati)
def generate_global_outliers(n_samples=300, n_outliers=10, n_features=2, random_state=42):
    np.random.seed(random_state)
    
    # Generare dati normali
    X = np.random.randn(n_samples - n_outliers, n_features)
    
    # Generare outlier
    outliers = np.random.uniform(low=-4, high=4, size=(n_outliers, n_features))
    # Assicurarsi che gli outlier siano sufficientemente distanti
    for i in range(n_outliers):
        # Scegliere una direzione casuale
        direction = np.random.randn(n_features)
        direction = direction / np.linalg.norm(direction)
        # Posizionare l'outlier a una distanza casuale (ma grande) nella direzione scelta
        outliers[i] = direction * np.random.uniform(4, 6)
    
    # Combinare dati normali e outlier
    X_combined = np.vstack([X, outliers])
    
    # Creare le etichette (0: normale, 1: anomalia)
    y = np.zeros(n_samples)
    y[n_samples - n_outliers:] = 1
    
    return X_combined, y

# Generare e visualizzare il dataset con anomalie globali
X_global, y_global = generate_global_outliers(n_samples=300, n_outliers=15)
plot_dataset(X_global, y_global, "Dataset con Anomalie Globali")

In [ ]:
# 2. Dataset con anomalie locali (outlier vicini alla massa principale ma in regioni a bassa densità)
def generate_local_outliers(n_samples=300, n_outliers=10, n_features=2, random_state=42):
    np.random.seed(random_state)
    
    # Generare dati normali da due cluster
    X1 = np.random.randn(n_samples // 2, n_features) + np.array([3, 3])
    X2 = np.random.randn(n_samples // 2 - n_outliers, n_features) + np.array([-3, -3])
    
    # Generare outlier nella regione tra i due cluster
    outliers = np.random.uniform(low=-1, high=1, size=(n_outliers, n_features))
    
    # Combinare dati normali e outlier
    X_combined = np.vstack([X1, X2, outliers])
    
    # Creare le etichette (0: normale, 1: anomalia)
    y = np.zeros(n_samples)
    y[n_samples - n_outliers:] = 1
    
    return X_combined, y

# Generare e visualizzare il dataset con anomalie locali
X_local, y_local = generate_local_outliers(n_samples=300, n_outliers=15)
plot_dataset(X_local, y_local, "Dataset con Anomalie Locali")

In [ ]:
# 3. Dataset con cluster di anomalie
def generate_clustered_outliers(n_samples=300, n_outliers=20, n_features=2, random_state=42):
    np.random.seed(random_state)
    
    # Generare dati normali
    X = np.random.randn(n_samples - n_outliers, n_features) * 1.5
    
    # Generare un piccolo cluster di outlier
    outliers = np.random.randn(n_outliers, n_features) * 0.3 + np.array([5, 5])
    
    # Combinare dati normali e outlier
    X_combined = np.vstack([X, outliers])
    
    # Creare le etichette (0: normale, 1: anomalia)
    y = np.zeros(n_samples)
    y[n_samples - n_outliers:] = 1
    
    return X_combined, y

# Generare e visualizzare il dataset con cluster di anomalie
X_clustered, y_clustered = generate_clustered_outliers(n_samples=300, n_outliers=20)
plot_dataset(X_clustered, y_clustered, "Dataset con Cluster di Anomalie")

In [ ]:
# 4. Dataset con anomalie in alta dimensionalità
def generate_high_dim_outliers(n_samples=300, n_outliers=15, n_features=10, random_state=42):
    np.random.seed(random_state)
    
    # Generare dati normali
    X = np.random.randn(n_samples - n_outliers, n_features)
    
    # Generare outlier con valori estremi in alcune dimensioni
    outliers = np.random.randn(n_outliers, n_features)
    for i in range(n_outliers):
        # Selezionare casualmente alcune dimensioni (1-3) per avere valori estremi
        n_extreme_dims = np.random.randint(1, 4)
        extreme_dims = np.random.choice(n_features, n_extreme_dims, replace=False)
        for dim in extreme_dims:
            # Assegnare un valore estremo (positivo o negativo)
            outliers[i, dim] = np.random.choice([-1, 1]) * np.random.uniform(5, 8)
    
    # Combinare dati normali e outlier
    X_combined = np.vstack([X, outliers])
    
    # Creare le etichette (0: normale, 1: anomalia)
    y = np.zeros(n_samples)
    y[n_samples - n_outliers:] = 1
    
    return X_combined, y

# Generare e visualizzare il dataset con anomalie in alta dimensionalità
X_high_dim, y_high_dim = generate_high_dim_outliers(n_samples=300, n_outliers=15, n_features=10)
plot_dataset(X_high_dim, y_high_dim, "Dataset con Anomalie in Alta Dimensionalità")

## Parte 2: Approcci Statistici al Rilevamento di Anomalie

Implementiamo e applichiamo approcci statistici per il rilevamento di anomalie.

### 2.1 Z-score

In [ ]:
def detect_anomalies_zscore(X, threshold=3.0):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Calcolare il z-score per ogni feature
    z_scores = np.abs(X_scaled)
    
    # Considerare il massimo z-score tra tutte le feature per ogni punto
    max_z_scores = np.max(z_scores, axis=1)
    
    # Identificare le anomalie
    anomalies = max_z_scores > threshold
    
    return anomalies, max_z_scores

In [ ]:
# Applicare Z-score al dataset con anomalie globali
anomalies_zscore, scores_zscore = detect_anomalies_zscore(X_global)

# Visualizzare i risultati
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_global[:, 0], X_global[:, 1], c=anomalies_zscore, cmap='coolwarm', s=50, alpha=0.7)
plt.colorbar(label='Anomalia')
plt.title('Rilevamento Anomalie con Z-score')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_global[:, 0], X_global[:, 1], c=scores_zscore, cmap='viridis', s=50, alpha=0.7)
plt.colorbar(label='Max Z-score')
plt.title('Punteggi Z-score')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Valutare le prestazioni
print("Matrice di confusione:")
print(confusion_matrix(y_global, anomalies_zscore))
print("\nReport di classificazione:")
print(classification_report(y_global, anomalies_zscore))

### 2.2 Elliptic Envelope (Mahalanobis Distance)

In [ ]:
def detect_anomalies_elliptic(X, contamination=0.05):
    # Applicare Elliptic Envelope
    detector = EllipticEnvelope(contamination=contamination, random_state=42)
    detector.fit(X)
    
    # Predire le anomalie (-1 per anomalie, 1 per normali)
    y_pred = detector.predict(X)
    anomalies = y_pred == -1
    
    # Calcolare i punteggi di anomalia
    scores = -detector.decision_function(X)  # Negativo per avere punteggi più alti per le anomalie
    
    return anomalies, scores

In [ ]:
# Applicare Elliptic Envelope al dataset con anomalie globali
contamination = len(y_global[y_global == 1]) / len(y_global)  # Proporzione reale di anomalie
anomalies_elliptic, scores_elliptic = detect_anomalies_elliptic(X_global, contamination=contamination)

# Visualizzare i risultati
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_global[:, 0], X_global[:, 1], c=anomalies_elliptic, cmap='coolwarm', s=50, alpha=0.7)
plt.colorbar(label='Anomalia')
plt.title('Rilevamento Anomalie con Elliptic Envelope')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_global[:, 0], X_global[:, 1], c=scores_elliptic, cmap='viridis', s=50, alpha=0.7)
plt.colorbar(label='Punteggio Anomalia')
plt.title('Punteggi Elliptic Envelope')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Valutare le prestazioni
print("Matrice di confusione:")
print(confusion_matrix(y_global, anomalies_elliptic))
print("\nReport di classificazione:")
print(classification_report(y_global, anomalies_elliptic))

### 2.3 Visualizzazione delle regioni di decisione

In [ ]:
def plot_decision_boundary(X, detector, title="Decision Boundary", method_name=""):
    # Creare una griglia di punti
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    
    # Calcolare i punteggi di anomalia per ogni punto della griglia
    if method_name == "Z-score":
        # Per Z-score, dobbiamo implementare manualmente
        scaler = StandardScaler()
        scaler.fit(X)
        grid_points = np.c_[xx.ravel(), yy.ravel()]
        grid_points_scaled = scaler.transform(grid_points)
        Z = np.max(np.abs(grid_points_scaled), axis=1)
        Z = Z.reshape(xx.shape)
        levels = [3.0]  # Soglia Z-score
    else:
        # Per altri metodi, utilizziamo decision_function
        Z = detector.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        if method_name == "Elliptic Envelope":
            levels = [0]  # Soglia Elliptic Envelope
        else:
            levels = [0]  # Soglia generica
    
    # Visualizzare la regione di decisione
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, levels=10, cmap='viridis', alpha=0.3)
    plt.colorbar(label='Punteggio Anomalia')
    plt.contour(xx, yy, Z, levels=levels, colors='red', linestyles='dashed')
    
    # Visualizzare i punti
    if hasattr(detector, 'predict'):
        y_pred = detector.predict(X)
        if method_name == "Elliptic Envelope":
            anomalies = y_pred == -1
        else:
            anomalies = y_pred == -1
    else:
        # Per Z-score
        anomalies, _ = detect_anomalies_zscore(X)
    
    plt.scatter(X[~anomalies, 0], X[~anomalies, 1], c='blue', label='Normale', s=50, alpha=0.7)
    plt.scatter(X[anomalies, 0], X[anomalies, 1], c='red', label='Anomalia', s=80, alpha=0.7, marker='X')
    
    plt.title(title, fontsize=14)
    plt.xlabel('Feature 1', fontsize=12)
    plt.ylabel('Feature 2', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualizzare la regione di decisione per Elliptic Envelope
detector_elliptic = EllipticEnvelope(contamination=contamination, random_state=42)
detector_elliptic.fit(X_global)
plot_decision_boundary(X_global, detector_elliptic, "Regione di Decisione - Elliptic Envelope", "Elliptic Envelope")

## Parte 3: Metodi Basati sulla Densità

Implementiamo e applichiamo metodi basati sulla densità per il rilevamento di anomalie.

### 3.1 Local Outlier Factor (LOF)

In [ ]:
def detect_anomalies_lof(X, n_neighbors=20, contamination=0.05):
    # Applicare Local Outlier Factor
    detector = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=False)
    
    # Predire le anomalie (-1 per anomalie, 1 per normali)
    y_pred = detector.fit_predict(X)
    anomalies = y_pred == -1
    
    # Calcolare i punteggi LOF
    scores = -detector.negative_outlier_factor_  # Negativo per avere punteggi più alti per le anomalie
    
    return anomalies, scores

In [ ]:
# Applicare LOF al dataset con anomalie locali
contamination_local = len(y_local[y_local == 1]) / len(y_local)  # Proporzione reale di anomalie
anomalies_lof, scores_lof = detect_anomalies_lof(X_local, contamination=contamination_local)

# Visualizzare i risultati
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_local[:, 0], X_local[:, 1], c=anomalies_lof, cmap='coolwarm', s=50, alpha=0.7)
plt.colorbar(label='Anomalia')
plt.title('Rilevamento Anomalie con LOF')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_local[:, 0], X_local[:, 1], c=scores_lof, cmap='viridis', s=50, alpha=0.7)
plt.colorbar(label='Punteggio LOF')
plt.title('Punteggi LOF')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Valutare le prestazioni
print("Matrice di confusione:")
print(confusion_matrix(y_local, anomalies_lof))
print("\nReport di classificazione:")
print(classification_report(y_local, anomalies_lof))

### 3.2 DBSCAN per il Rilevamento di Anomalie

In [ ]:
def detect_anomalies_dbscan(X, eps=0.5, min_samples=5):
    # Applicare DBSCAN
    detector = DBSCAN(eps=eps, min_samples=min_samples)
    detector.fit(X)
    
    # Predire le anomalie (-1 per rumore/anomalie)
    labels = detector.labels_
    anomalies = labels == -1
    
    # Calcolare una sorta di punteggio di anomalia basato sulla distanza dal cluster più vicino
    # Questo è un approccio semplificato, non è un punteggio standard di DBSCAN
    scores = np.zeros(X.shape[0])
    
    # Per ogni punto, calcolare la distanza minima dai core points
    core_samples_mask = np.zeros_like(labels, dtype=bool)
    core_samples_mask[detector.core_sample_indices_] = True
    
    if np.any(core_samples_mask):
        # Se ci sono core points, calcolare la distanza da essi
        core_points = X[core_samples_mask]
        for i, x in enumerate(X):
            min_dist = np.min(np.linalg.norm(x - core_points, axis=1))
            scores[i] = min_dist
    
    return anomalies, scores

In [ ]:
# Determinare i parametri ottimali per DBSCAN
def find_optimal_eps(X, min_samples=5, k=5):
    # Calcolare le distanze ai k vicini più prossimi
    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(X)
    distances, indices = neigh.kneighbors(X)
    
    # Ordinare le distanze in ordine crescente
    distances = np.sort(distances[:, k-1])
    
    # Visualizzare il grafico delle distanze
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(distances)), distances, 'b-')
    plt.xlabel('Punti ordinati per distanza')
    plt.ylabel(f'Distanza al {k}-esimo vicino più prossimo')
    plt.title('Grafico delle distanze per determinare eps ottimale')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Calcolare la derivata per trovare il punto di massima curvatura
    derivative = np.gradient(distances)
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(derivative)), derivative, 'r-')
    plt.xlabel('Punti ordinati per distanza')
    plt.ylabel('Derivata della distanza')
    plt.title('Derivata delle distanze per identificare il "gomito"')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Trovare il punto di massima curvatura (approssimazione)
    knee_point = np.argmax(derivative) if len(derivative) > 0 else 0
    suggested_eps = distances[knee_point]
    
    print(f"Valore di eps suggerito: {suggested_eps:.3f}")
    print(f"Valore di min_samples suggerito: {min_samples}")
    
    return suggested_eps, min_samples

In [ ]:
# Trovare i parametri ottimali per DBSCAN sul dataset con anomalie locali
eps_local, min_samples_local = find_optimal_eps(X_local)

In [ ]:
# Applicare DBSCAN al dataset con anomalie locali
anomalies_dbscan, scores_dbscan = detect_anomalies_dbscan(X_local, eps=eps_local, min_samples=min_samples_local)

# Visualizzare i risultati
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_local[:, 0], X_local[:, 1], c=anomalies_dbscan, cmap='coolwarm', s=50, alpha=0.7)
plt.colorbar(label='Anomalia')
plt.title('Rilevamento Anomalie con DBSCAN')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_local[:, 0], X_local[:, 1], c=scores_dbscan, cmap='viridis', s=50, alpha=0.7)
plt.colorbar(label='Distanza dal cluster più vicino')
plt.title('Punteggi DBSCAN')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Valutare le prestazioni
print("Matrice di confusione:")
print(confusion_matrix(y_local, anomalies_dbscan))
print("\nReport di classificazione:")
print(classification_report(y_local, anomalies_dbscan))

## Parte 4: Isolation Forest

Implementiamo e applichiamo Isolation Forest per il rilevamento di anomalie.

In [ ]:
def detect_anomalies_iforest(X, n_estimators=100, contamination=0.05, max_samples='auto'):
    # Applicare Isolation Forest
    detector = IsolationForest(n_estimators=n_estimators, contamination=contamination, 
                              max_samples=max_samples, random_state=42)
    detector.fit(X)
    
    # Predire le anomalie (-1 per anomalie, 1 per normali)
    y_pred = detector.predict(X)
    anomalies = y_pred == -1
    
    # Calcolare i punteggi di anomalia
    scores = -detector.decision_function(X)  # Negativo per avere punteggi più alti per le anomalie
    
    return anomalies, scores, detector

In [ ]:
# Applicare Isolation Forest al dataset con anomalie globali
anomalies_iforest, scores_iforest, detector_iforest = detect_anomalies_iforest(X_global, contamination=contamination)

# Visualizzare i risultati
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_global[:, 0], X_global[:, 1], c=anomalies_iforest, cmap='coolwarm', s=50, alpha=0.7)
plt.colorbar(label='Anomalia')
plt.title('Rilevamento Anomalie con Isolation Forest')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_global[:, 0], X_global[:, 1], c=scores_iforest, cmap='viridis', s=50, alpha=0.7)
plt.colorbar(label='Punteggio Anomalia')
plt.title('Punteggi Isolation Forest')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Valutare le prestazioni
print("Matrice di confusione:")
print(confusion_matrix(y_global, anomalies_iforest))
print("\nReport di classificazione:")
print(classification_report(y_global, anomalies_iforest))

In [ ]:
# Visualizzare la regione di decisione per Isolation Forest
plot_decision_boundary(X_global, detector_iforest, "Regione di Decisione - Isolation Forest", "Isolation Forest")

### 4.1 Effetto dei parametri di Isolation Forest

In [ ]:
def explore_iforest_parameters(X, y, n_estimators_list=[50, 100, 200], max_samples_list=['auto', 64, 128]):
    # Calcolare la contaminazione reale
    contamination = np.sum(y == 1) / len(y)
    
    # Preparare la figura
    fig, axes = plt.subplots(len(n_estimators_list), len(max_samples_list), figsize=(15, 12))
    
    # Per ogni combinazione di parametri
    for i, n_estimators in enumerate(n_estimators_list):
        for j, max_samples in enumerate(max_samples_list):
            # Applicare Isolation Forest
            detector = IsolationForest(n_estimators=n_estimators, contamination=contamination, 
                                      max_samples=max_samples, random_state=42)
            detector.fit(X)
            
            # Predire le anomalie
            y_pred = detector.predict(X)
            anomalies = y_pred == -1
            
            # Calcolare le metriche
            precision = precision_score(y, anomalies)
            recall = recall_score(y, anomalies)
            f1 = f1_score(y, anomalies)
            
            # Visualizzare i risultati
            axes[i, j].scatter(X[:, 0], X[:, 1], c=anomalies, cmap='coolwarm', s=30, alpha=0.7)
            axes[i, j].set_title(f"n_est={n_estimators}, max_samp={max_samples}\nP={precision:.2f}, R={recall:.2f}, F1={f1:.2f}")
            axes[i, j].set_xlabel('Feature 1')
            axes[i, j].set_ylabel('Feature 2')
            axes[i, j].grid(True, alpha=0.3)
    
    plt.suptitle("Effetto dei parametri in Isolation Forest", fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

In [ ]:
# Esplorare l'effetto dei parametri di Isolation Forest sul dataset con anomalie globali
explore_iforest_parameters(X_global, y_global)

## Parte 5: One-Class SVM

Implementiamo e applichiamo One-Class SVM per il rilevamento di anomalie.

In [ ]:
def detect_anomalies_ocsvm(X, nu=0.05, kernel='rbf', gamma='scale'):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Applicare One-Class SVM
    detector = OneClassSVM(nu=nu, kernel=kernel, gamma=gamma)
    detector.fit(X_scaled)
    
    # Predire le anomalie (-1 per anomalie, 1 per normali)
    y_pred = detector.predict(X_scaled)
    anomalies = y_pred == -1
    
    # Calcolare i punteggi di anomalia
    scores = -detector.decision_function(X_scaled)  # Negativo per avere punteggi più alti per le anomalie
    
    return anomalies, scores, detector, X_scaled

In [ ]:
# Applicare One-Class SVM al dataset con anomalie globali
anomalies_ocsvm, scores_ocsvm, detector_ocsvm, X_global_scaled = detect_anomalies_ocsvm(X_global, nu=contamination)

# Visualizzare i risultati
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_global[:, 0], X_global[:, 1], c=anomalies_ocsvm, cmap='coolwarm', s=50, alpha=0.7)
plt.colorbar(label='Anomalia')
plt.title('Rilevamento Anomalie con One-Class SVM')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_global[:, 0], X_global[:, 1], c=scores_ocsvm, cmap='viridis', s=50, alpha=0.7)
plt.colorbar(label='Punteggio Anomalia')
plt.title('Punteggi One-Class SVM')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Valutare le prestazioni
print("Matrice di confusione:")
print(confusion_matrix(y_global, anomalies_ocsvm))
print("\nReport di classificazione:")
print(classification_report(y_global, anomalies_ocsvm))

In [ ]:
# Visualizzare la regione di decisione per One-Class SVM
plot_decision_boundary(X_global_scaled, detector_ocsvm, "Regione di Decisione - One-Class SVM", "One-Class SVM")

### 5.1 Effetto dei parametri di One-Class SVM

In [ ]:
def explore_ocsvm_parameters(X, y, nu_list=[0.01, 0.05, 0.1], gamma_list=['scale', 0.1, 1.0]):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Preparare la figura
    fig, axes = plt.subplots(len(nu_list), len(gamma_list), figsize=(15, 12))
    
    # Per ogni combinazione di parametri
    for i, nu in enumerate(nu_list):
        for j, gamma in enumerate(gamma_list):
            # Applicare One-Class SVM
            detector = OneClassSVM(nu=nu, kernel='rbf', gamma=gamma)
            detector.fit(X_scaled)
            
            # Predire le anomalie
            y_pred = detector.predict(X_scaled)
            anomalies = y_pred == -1
            
            # Calcolare le metriche
            precision = precision_score(y, anomalies)
            recall = recall_score(y, anomalies)
            f1 = f1_score(y, anomalies)
            
            # Visualizzare i risultati
            axes[i, j].scatter(X[:, 0], X[:, 1], c=anomalies, cmap='coolwarm', s=30, alpha=0.7)
            axes[i, j].set_title(f"nu={nu}, gamma={gamma}\nP={precision:.2f}, R={recall:.2f}, F1={f1:.2f}")
            axes[i, j].set_xlabel('Feature 1')
            axes[i, j].set_ylabel('Feature 2')
            axes[i, j].grid(True, alpha=0.3)
    
    plt.suptitle("Effetto dei parametri in One-Class SVM", fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

In [ ]:
# Esplorare l'effetto dei parametri di One-Class SVM sul dataset con anomalie globali
explore_ocsvm_parameters(X_global, y_global)

## Parte 6: Confronto tra Algoritmi di Rilevamento di Anomalie

Confrontiamo le prestazioni dei diversi algoritmi di rilevamento di anomalie sui nostri dataset.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

def compare_anomaly_detection_algorithms(X, y, contamination=None):
    if contamination is None:
        contamination = np.sum(y == 1) / len(y)
    
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Definire gli algoritmi da confrontare
    algorithms = {
        'Z-score': None,  # Implementazione manuale
        'Elliptic Envelope': EllipticEnvelope(contamination=contamination, random_state=42),
        'Local Outlier Factor': LocalOutlierFactor(n_neighbors=20, contamination=contamination, novelty=False),
        'Isolation Forest': IsolationForest(n_estimators=100, contamination=contamination, random_state=42),
        'One-Class SVM': OneClassSVM(nu=contamination, kernel='rbf', gamma='scale')
    }
    
    results = {}
    
    # Applicare ogni algoritmo
    for name, algorithm in algorithms.items():
        start_time = time.time()
        
        if name == 'Z-score':
            # Implementazione manuale di Z-score
            anomalies, scores = detect_anomalies_zscore(X, threshold=3.0)
        elif name == 'Local Outlier Factor':
            # LOF in modalità non-novelty
            y_pred = algorithm.fit_predict(X_scaled)
            anomalies = y_pred == -1
            scores = -algorithm.negative_outlier_factor_
        else:
            # Altri algoritmi
            if name == 'One-Class SVM':
                algorithm.fit(X_scaled)
                y_pred = algorithm.predict(X_scaled)
                scores = -algorithm.decision_function(X_scaled)
            else:
                algorithm.fit(X_scaled)
                y_pred = algorithm.predict(X_scaled)
                scores = -algorithm.decision_function(X_scaled) if hasattr(algorithm, 'decision_function') else np.zeros(len(X))
            anomalies = y_pred == -1
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        # Calcolare le metriche
        precision = precision_score(y, anomalies)
        recall = recall_score(y, anomalies)
        f1 = f1_score(y, anomalies)
        
        # Calcolare AUC se possibile
        try:
            auc = roc_auc_score(y, scores)
        except:
            auc = np.nan
        
        results[name] = {
            'anomalies': anomalies,
            'scores': scores,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc,
            'time': execution_time
        }
    
    # Visualizzare i risultati
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    # Visualizzare il dataset originale con le anomalie vere
    axes[0].scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', s=50, alpha=0.7)
    axes[0].set_title('Anomalie Vere')
    axes[0].set_xlabel('Feature 1')
    axes[0].set_ylabel('Feature 2')
    axes[0].grid(True, alpha=0.3)
    
    # Visualizzare i risultati di ogni algoritmo
    for i, (name, result) in enumerate(results.items(), 1):
        if i < len(axes):
            axes[i].scatter(X[:, 0], X[:, 1], c=result['anomalies'], cmap='coolwarm', s=50, alpha=0.7)
            axes[i].set_title(f"{name}\nP={result['precision']:.2f}, R={result['recall']:.2f}, F1={result['f1']:.2f}")
            axes[i].set_xlabel('Feature 1')
            axes[i].set_ylabel('Feature 2')
            axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Visualizzare le metriche di valutazione
    metrics = ['precision', 'recall', 'f1', 'auc', 'time']
    metrics_df = pd.DataFrame({name: [result[metric] for metric in metrics] for name, result in results.items()}, index=metrics)
    
    # Visualizzare le metriche come grafico a barre
    plt.figure(figsize=(12, 8))
    
    # Visualizzare precision, recall, f1, auc
    plt.subplot(2, 1, 1)
    metrics_df.loc[['precision', 'recall', 'f1', 'auc']].plot(kind='bar', ax=plt.gca())
    plt.title('Metriche di Valutazione')
    plt.ylabel('Valore')
    plt.ylim(0, 1.1)
    plt.grid(True, alpha=0.3, axis='y')
    
    # Visualizzare il tempo di esecuzione
    plt.subplot(2, 1, 2)
    metrics_df.loc['time'].plot(kind='bar', ax=plt.gca(), color='green')
    plt.title('Tempo di Esecuzione')
    plt.ylabel('Secondi')
    plt.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    return results, metrics_df

In [ ]:
# Confrontare gli algoritmi sul dataset con anomalie globali
results_global, metrics_global = compare_anomaly_detection_algorithms(X_global, y_global)

In [ ]:
# Confrontare gli algoritmi sul dataset con anomalie locali
results_local, metrics_local = compare_anomaly_detection_algorithms(X_local, y_local)

In [ ]:
# Confrontare gli algoritmi sul dataset con cluster di anomalie
results_clustered, metrics_clustered = compare_anomaly_detection_algorithms(X_clustered, y_clustered)

### 6.1 Curve ROC e Precision-Recall

In [ ]:
def plot_roc_pr_curves(results, y_true):
    plt.figure(figsize=(12, 5))
    
    # Curva ROC
    plt.subplot(1, 2, 1)
    for name, result in results.items():
        if not np.isnan(result['auc']):
            fpr, tpr, _ = roc_curve(y_true, result['scores'])
            plt.plot(fpr, tpr, label=f"{name} (AUC = {result['auc']:.3f})")
    
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curve ROC')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    
    # Curva Precision-Recall
    plt.subplot(1, 2, 2)
    for name, result in results.items():
        if not np.isnan(result['auc']):
            precision, recall, _ = precision_recall_curve(y_true, result['scores'])
            avg_precision = average_precision_score(y_true, result['scores'])
            plt.plot(recall, precision, label=f"{name} (AP = {avg_precision:.3f})")
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Curve Precision-Recall')
    plt.legend(loc='lower left')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualizzare le curve ROC e Precision-Recall per il dataset con anomalie globali
plot_roc_pr_curves(results_global, y_global)

## Parte 7: Rilevamento di Anomalie in Alta Dimensionalità

Esploriamo il rilevamento di anomalie in dataset ad alta dimensionalità.

In [ ]:
# Confrontare gli algoritmi sul dataset con anomalie in alta dimensionalità
results_high_dim, metrics_high_dim = compare_anomaly_detection_algorithms(X_high_dim, y_high_dim)

In [ ]:
# Visualizzare le curve ROC e Precision-Recall per il dataset con anomalie in alta dimensionalità
plot_roc_pr_curves(results_high_dim, y_high_dim)

### 7.1 Riduzione della dimensionalità prima del rilevamento di anomalie

In [ ]:
def detect_anomalies_with_dim_reduction(X, y, n_components=2, contamination=None):
    if contamination is None:
        contamination = np.sum(y == 1) / len(y)
    
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Applicare PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # Visualizzare i dati ridotti
    plt.figure(figsize=(10, 8))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='coolwarm', s=50, alpha=0.7)
    plt.colorbar(label='Anomalia')
    plt.title(f'Dati ridotti con PCA (varianza spiegata: {pca.explained_variance_ratio_.sum():.3f})')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Applicare gli algoritmi di rilevamento anomalie sui dati ridotti
    results_pca, metrics_pca = compare_anomaly_detection_algorithms(X_pca, y, contamination)
    
    return results_pca, metrics_pca, X_pca

In [ ]:
# Applicare la riduzione della dimensionalità prima del rilevamento di anomalie
results_high_dim_pca, metrics_high_dim_pca, X_high_dim_pca = detect_anomalies_with_dim_reduction(X_high_dim, y_high_dim, n_components=2)

In [ ]:
# Confrontare le prestazioni prima e dopo la riduzione della dimensionalità
plt.figure(figsize=(12, 6))

# Confrontare F1-score
plt.subplot(1, 2, 1)
f1_original = metrics_high_dim.loc['f1']
f1_pca = metrics_high_dim_pca.loc['f1']
algorithms = f1_original.index

x = np.arange(len(algorithms))
width = 0.35

plt.bar(x - width/2, f1_original, width, label='Originale')
plt.bar(x + width/2, f1_pca, width, label='Dopo PCA')

plt.xlabel('Algoritmo')
plt.ylabel('F1-score')
plt.title('F1-score prima e dopo PCA')
plt.xticks(x, algorithms, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

# Confrontare AUC
plt.subplot(1, 2, 2)
auc_original = metrics_high_dim.loc['auc']
auc_pca = metrics_high_dim_pca.loc['auc']

plt.bar(x - width/2, auc_original, width, label='Originale')
plt.bar(x + width/2, auc_pca, width, label='Dopo PCA')

plt.xlabel('Algoritmo')
plt.ylabel('AUC')
plt.title('AUC prima e dopo PCA')
plt.xticks(x, algorithms, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Parte 8: Rilevamento di Anomalie in Serie Temporali

Esploriamo il rilevamento di anomalie in serie temporali.

In [ ]:
# Generare una serie temporale sintetica con anomalie
def generate_time_series_with_anomalies(n_samples=1000, n_anomalies=50, anomaly_std=5, random_state=42):
    np.random.seed(random_state)
    
    # Generare il tempo
    t = np.linspace(0, 10, n_samples)
    
    # Generare il segnale normale (sinusoidale con rumore)
    signal = 10 * np.sin(t) + np.random.normal(0, 1, n_samples)
    
    # Aggiungere un trend
    trend = 0.01 * t**2
    signal = signal + trend
    
    # Aggiungere stagionalità
    seasonality = 5 * np.sin(t * 10)
    signal = signal + seasonality
    
    # Creare le etichette (0: normale, 1: anomalia)
    y = np.zeros(n_samples)
    
    # Aggiungere anomalie puntuali
    anomaly_indices = np.random.choice(n_samples, n_anomalies, replace=False)
    signal[anomaly_indices] += np.random.normal(0, anomaly_std, n_anomalies)
    y[anomaly_indices] = 1
    
    return t, signal, y

In [ ]:
# Generare e visualizzare la serie temporale
t, signal, y_ts = generate_time_series_with_anomalies()

plt.figure(figsize=(12, 6))
plt.plot(t, signal, 'b-', alpha=0.7, label='Segnale')
plt.scatter(t[y_ts == 1], signal[y_ts == 1], color='red', s=50, label='Anomalie')
plt.title('Serie Temporale con Anomalie')
plt.xlabel('Tempo')
plt.ylabel('Valore')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 8.1 Approccio statistico: Z-score su finestra mobile

In [ ]:
def detect_anomalies_rolling_zscore(signal, window_size=50, threshold=3.0):
    # Inizializzare l'array dei punteggi
    scores = np.zeros_like(signal)
    
    # Calcolare lo z-score su finestra mobile
    for i in range(len(signal)):
        if i < window_size:
            # Per i primi punti, usare tutti i dati disponibili
            window = signal[:i+1]
        else:
            # Altrimenti, usare la finestra mobile
            window = signal[i-window_size+1:i+1]
        
        if len(window) > 1:  # Assicurarsi che ci siano abbastanza dati per calcolare media e std
            mean = np.mean(window)
            std = np.std(window)
            if std > 0:  # Evitare divisione per zero
                scores[i] = abs((signal[i] - mean) / std)
    
    # Identificare le anomalie
    anomalies = scores > threshold
    
    return anomalies, scores

In [ ]:
# Applicare Z-score su finestra mobile
anomalies_rolling, scores_rolling = detect_anomalies_rolling_zscore(signal)

# Visualizzare i risultati
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(t, signal, 'b-', alpha=0.7, label='Segnale')
plt.scatter(t[anomalies_rolling], signal[anomalies_rolling], color='red', s=50, label='Anomalie rilevate')
plt.scatter(t[y_ts == 1], signal[y_ts == 1], color='green', s=30, alpha=0.5, label='Anomalie vere')
plt.title('Rilevamento Anomalie con Z-score su Finestra Mobile')
plt.xlabel('Tempo')
plt.ylabel('Valore')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(t, scores_rolling, 'r-')
plt.axhline(y=3.0, color='k', linestyle='--', alpha=0.7, label='Soglia (Z=3)')
plt.title('Punteggi Z-score')
plt.xlabel('Tempo')
plt.ylabel('Z-score')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Valutare le prestazioni
print("Matrice di confusione:")
print(confusion_matrix(y_ts, anomalies_rolling))
print("\nReport di classificazione:")
print(classification_report(y_ts, anomalies_rolling))

### 8.2 Isolation Forest per serie temporali

In [ ]:
def detect_anomalies_ts_iforest(signal, window_size=10, contamination=0.05):
    # Creare feature da finestre temporali
    X = np.zeros((len(signal) - window_size + 1, window_size))
    for i in range(len(X)):
        X[i] = signal[i:i+window_size]
    
    # Applicare Isolation Forest
    detector = IsolationForest(contamination=contamination, random_state=42)
    detector.fit(X)
    
    # Predire le anomalie (-1 per anomalie, 1 per normali)
    y_pred = detector.predict(X)
    anomalies_window = y_pred == -1
    
    # Calcolare i punteggi di anomalia
    scores_window = -detector.decision_function(X)  # Negativo per avere punteggi più alti per le anomalie
    
    # Convertire i risultati alla lunghezza originale della serie
    anomalies = np.zeros(len(signal), dtype=bool)
    scores = np.zeros(len(signal))
    
    # Assegnare l'etichetta di anomalia se almeno una finestra che contiene il punto è anomala
    for i in range(len(signal)):
        # Trovare tutte le finestre che contengono il punto i
        window_indices = [j for j in range(max(0, i - window_size + 1), min(i + 1, len(anomalies_window)))]
        if window_indices:  # Se ci sono finestre che contengono il punto
            # Punto è anomalo se almeno una finestra che lo contiene è anomala
            anomalies[i] = np.any(anomalies_window[window_indices])
            # Punteggio è il massimo tra i punteggi delle finestre che contengono il punto
            scores[i] = np.max(scores_window[window_indices])
    
    return anomalies, scores

In [ ]:
# Applicare Isolation Forest per serie temporali
contamination_ts = np.sum(y_ts == 1) / len(y_ts)
anomalies_ts_iforest, scores_ts_iforest = detect_anomalies_ts_iforest(signal, contamination=contamination_ts)

# Visualizzare i risultati
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(t, signal, 'b-', alpha=0.7, label='Segnale')
plt.scatter(t[anomalies_ts_iforest], signal[anomalies_ts_iforest], color='red', s=50, label='Anomalie rilevate')
plt.scatter(t[y_ts == 1], signal[y_ts == 1], color='green', s=30, alpha=0.5, label='Anomalie vere')
plt.title('Rilevamento Anomalie con Isolation Forest per Serie Temporali')
plt.xlabel('Tempo')
plt.ylabel('Valore')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(t, scores_ts_iforest, 'r-')
plt.title('Punteggi Isolation Forest')
plt.xlabel('Tempo')
plt.ylabel('Punteggio Anomalia')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Valutare le prestazioni
print("Matrice di confusione:")
print(confusion_matrix(y_ts, anomalies_ts_iforest))
print("\nReport di classificazione:")
print(classification_report(y_ts, anomalies_ts_iforest))

## Parte 9: Caso di Studio - Rilevamento di Frodi nelle Transazioni con Carte di Credito

Applichiamo le tecniche di rilevamento di anomalie a un caso di studio reale: il rilevamento di frodi nelle transazioni con carte di credito.

In [ ]:
# Caricare il dataset Credit Card Fraud Detection
# Nota: questo dataset è molto grande, quindi utilizziamo solo un sottoinsieme
from sklearn.datasets import fetch_openml

# Scaricare il dataset (potrebbe richiedere tempo)
try:
    cc_fraud = fetch_openml(name='credit-card-fraud', version=1, parser='auto')
    X_cc = cc_fraud.data.values
    y_cc = cc_fraud.target.astype(int).values
    
    # Prendere solo un sottoinsieme per velocizzare l'esercitazione
    n_samples = 5000
    indices = np.random.choice(len(X_cc), n_samples, replace=False)
    X_cc = X_cc[indices]
    y_cc = y_cc[indices]
    
    print(f"Dataset Credit Card Fraud: {X_cc.shape[0]} campioni, {X_cc.shape[1]} features")
    print(f"Numero di frodi: {np.sum(y_cc == 1)} ({np.sum(y_cc == 1) / len(y_cc) * 100:.2f}%)")
    
    # Visualizzare il dataset
    plot_dataset(X_cc, y_cc, "Dataset Credit Card Fraud")
    
    # Confrontare gli algoritmi
    results_cc, metrics_cc = compare_anomaly_detection_algorithms(X_cc, y_cc)
    
    # Visualizzare le curve ROC e Precision-Recall
    plot_roc_pr_curves(results_cc, y_cc)
    
except Exception as e:
    print(f"Errore nel caricamento del dataset: {e}")
    print("Utilizziamo un dataset sintetico come alternativa.")
    
    # Generare un dataset sintetico che simula frodi
    X_cc, y_cc = generate_high_dim_outliers(n_samples=1000, n_outliers=20, n_features=10, random_state=42)
    
    print(f"Dataset sintetico: {X_cc.shape[0]} campioni, {X_cc.shape[1]} features")
    print(f"Numero di frodi: {np.sum(y_cc == 1)} ({np.sum(y_cc == 1) / len(y_cc) * 100:.2f}%)")
    
    # Visualizzare il dataset
    plot_dataset(X_cc, y_cc, "Dataset Sintetico (Simulazione Frodi)")
    
    # Confrontare gli algoritmi
    results_cc, metrics_cc = compare_anomaly_detection_algorithms(X_cc, y_cc)
    
    # Visualizzare le curve ROC e Precision-Recall
    plot_roc_pr_curves(results_cc, y_cc)

## Conclusioni

In questa esercitazione pratica, abbiamo esplorato diverse tecniche di rilevamento di anomalie e le abbiamo applicate a vari dataset. Abbiamo visto come:

1. **Approcci statistici** come Z-score ed Elliptic Envelope sono semplici ed efficaci per anomalie globali, ma possono avere difficoltà con distribuzioni complesse.
2. **Metodi basati sulla densità** come LOF e DBSCAN eccellono nell'identificare anomalie locali in regioni a bassa densità.
3. **Isolation Forest** è efficiente e scalabile, particolarmente adatto per dataset ad alta dimensionalità.
4. **One-Class SVM** può catturare confini di decisione complessi, ma è sensibile alla scelta dei parametri.
5. **Tecniche per serie temporali** richiedono approcci specifici che tengano conto della dipendenza temporale.

Abbiamo anche imparato l'importanza di:
- Scegliere l'algoritmo appropriato in base alla natura delle anomalie e dei dati
- Ottimizzare i parametri degli algoritmi per ottenere i migliori risultati
- Valutare le prestazioni con metriche appropriate come precision, recall, F1-score e AUC
- Considerare la riduzione della dimensionalità come preprocessing per dataset ad alta dimensionalità

Il rilevamento di anomalie è un campo vasto con numerose applicazioni pratiche, dalla sicurezza informatica alla finanza, dalla medicina all'industria. La scelta dell'approccio giusto dipende fortemente dal contesto specifico e dalla natura dei dati.

## Esercizi Aggiuntivi

1. Implementa un ensemble di metodi di rilevamento di anomalie e confronta le prestazioni con i singoli algoritmi.
2. Applica le tecniche di rilevamento di anomalie a un dataset reale di tua scelta (ad esempio, dati di rete, log di sistema, dati medici).
3. Implementa un autoencoder per il rilevamento di anomalie e confrontalo con gli altri metodi.
4. Esplora l'effetto della normalizzazione dei dati sulle prestazioni dei diversi algoritmi di rilevamento di anomalie.
5. Implementa una versione semplificata di Isolation Forest da zero (senza utilizzare scikit-learn) e confronta i risultati con l'implementazione di scikit-learn.